In [1]:
import os
os.chdir('C://Users//17731//PycharmProjects//transformer_test_github//transformer_test')
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [2]:
from datasets import load_dataset

# 加载WMT 2014 英语-德语数据集
dataset = load_dataset('wmt14', 'de-en')

# 查看数据集的结构
print(dataset)


C:\ProgramData\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})


In [3]:
from utils import prepare_training_data

prepare_training_data(dataset, 'WMT_de_en')

In [8]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train('--input=data/WMT_de_en_train.txt --model_prefix=bpe --vocab_size=37000 --model_type=bpe')

In [6]:
from utils import generate_padding_mask, generate_future_mask, combine_padding_mask
import torch

batch_size = 2
seq_length = 5
source = torch.tensor([
    [1, 2, 0, 0, 0],
    [4, 5, 6, 0, 0]
])

target = torch.tensor([
    [1, 2, 4, 0, 0],
    [4, 5, 6, 9, 0]
])




In [18]:
padding_mask = (source == 0).unsqueeze(1).unsqueeze(2)
padding_mask = padding_mask.expand(-1, -1, padding_mask.shape[3], -1)
padding_mask

tensor([[[[False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True]]],


        [[[False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True]]]])

In [5]:
padding_mask = (source == 0).unsqueeze(1).unsqueeze(2)
padding_mask.shape
padding_mask = padding_mask | padding_mask.transpose(-2, -1)
padding_mask.shape

padding_mask2 = (target == 0).unsqueeze(1).unsqueeze(2)
padding_mask2 = padding_mask2 | padding_mask2.transpose(-2, -1)

a = padding_mask2[:, :, 0, :].unsqueeze(3)
a.shape
b = padding_mask[:, :, :, 0].unsqueeze(2)
c = a | b
c


tensor([[[[False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True]]],


        [[[False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [ True,  True,  True,  True,  True]]]])

In [6]:
size = 5
future_mask = torch.triu(torch.ones(size, size, dtype=torch.bool), diagonal=1)
future_mask

tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])

In [7]:
from models import MachineTranslation
import torch.nn as nn
from models.positional_encoding import PositionalEncoding
d_model = 512
n_head = 8
n_encoder_layers = 6
n_decoder_layers = 6
d_feedforward = 2048
dropout = 0.1
lr = 0.01
batch_size = 32
pad_token = 0
source_vocab_dim = 7
target_vocab_dim = 7

source = torch.tensor([
    [1, 2, 0, 0, 0],
    [4, 5, 6, 0, 0]
])

target = torch.tensor([
    [1, 2, 4, 0, 0],
    [4, 5, 6, 3, 0]
])

# model = MachineTranslation(source_vocab_dim, target_vocab_dim, d_model, n_head, n_encoder_layers, n_decoder_layers, d_feedforward, pad_token,
#                  dropout)
# output = model(source, target)
source_embedding = nn.Embedding(source_vocab_dim, d_model)
em = source_embedding(source)
pe = PositionalEncoding(d_model)
output = pe(em)
output

tensor([[[ 0.5084,  0.7560, -0.6812,  ..., -1.3319, -2.4242,  1.7258],
         [-0.4914,  1.5277,  2.0529,  ...,  1.5165,  0.0207,  0.3515],
         [ 1.6067, -0.0600, -0.5850,  ..., -0.0786, -0.3798, -0.6680],
         [ 1.6067, -0.0600, -0.5850,  ..., -0.0786, -0.3798, -0.6680],
         [ 1.6067, -0.0600, -0.5850,  ..., -0.0786, -0.3798, -0.6680]],

        [[-0.2418,  0.6281,  0.7402,  ...,  1.0797, -1.2930,  2.0868],
         [ 1.2005,  1.1456, -1.0344,  ...,  1.7965, -0.2928,  1.0111],
         [-0.4898, -1.4720,  1.8794,  ...,  0.2152,  1.2385, -0.2593],
         [ 2.4482, -0.5197,  0.2368,  ..., -0.0786, -0.3797, -0.6680],
         [ 2.4482, -0.5197,  0.2368,  ..., -0.0786, -0.3797, -0.6680]]],
       grad_fn=<AddBackward0>)